In [5]:
import os
import cv2 as cv
import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential , load_model
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
###################################################### 실행 안함. 자동완성용

from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping
######################################################

In [6]:

folder_list = os.listdir('/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train')
folder_list.remove('.DS_Store') 

data = []
pic_count = []

for folder_name in folder_list:
    folder_path = '/Users/jsp/Desktop/workspace/projects/fstteamproject/main/git_teamprj/proj1/open/train/' + folder_name
  
    image_list = os.listdir(folder_path)
    list_len = len(image_list)
    pic_count.append(list_len)
    
    for image_name in image_list:
        image_path = folder_path +'/' + image_name
        img = cv.imread(image_path)
        resize = cv.resize(img, (300,300), interpolation=cv.INTER_CUBIC)
        gray = cv.cvtColor(resize, cv.COLOR_BGR2GRAY)
        data.append(gray)

In [7]:
answer = []
for idx, nm in enumerate(folder_list):
        answer += [nm] * pic_count[idx]
answer = np.array(answer)
answer = pd.get_dummies(answer)
answer = answer.values

In [8]:
answer


array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True, False, False]])

In [9]:
xtrain, xtest, ytrain, ytest = train_test_split(np.array(data),answer,test_size=0.2, shuffle=True)

In [10]:
print(xtrain.shape)
print(xtest.shape)

(2765, 300, 300)
(692, 300, 300)


In [12]:
xtrain = xtrain.reshape(2765,300,300,1) 
xtest = xtest.reshape(692,300,300,1)

In [13]:
model = Sequential([
            Input(shape = (300,300,1), name = 'input_layer'),
            Conv2D(32,kernel_size=3, activation= 'relu', name = 'cov_layer1' ),
            #Dropout(0.5) 데이터 기반한 weight를 줄여 오버핏팅을 줄이는 것
            MaxPooling2D(pool_size= 2), 
            Flatten()
            ])

model.add(Dense(32, activation= 'relu'))
model.add(Dense(24, activation= 'relu'))
model.add(Dense(19, activation= 'softmax'))

model.summary()
model.compile(optimizer= 'adam', loss='categorical_crossentropy',metrics=['accuracy'])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cov_layer1 (Conv2D)         (None, 298, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 32)     0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 710432)            0         
                                                                 
 dense (Dense)               (None, 32)                22733856  
                                                                 
 dense_1 (Dense)             (None, 24)                792       
                                                                 
 dense_2 (Dense)             (None, 19)  

In [14]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)
modelpath="bestmodel.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

In [15]:

history = model.fit(xtrain,ytrain, validation_data=(xtest, ytest), batch_size=50, epochs = 500, verbose=1, callbacks=[early_stopping_callback,checkpointer])

Epoch 1/500


2023-05-08 15:03:14.693855: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


56/56 [==============================] - 9s 143ms/step - loss: 539.1890 - accuracy: 0.1834 - val_loss: 3.9950 - val_accuracy: 0.3382
Epoch 2/500
56/56 [==============================] - 7s 132ms/step - loss: 3.1139 - accuracy: 0.3186 - val_loss: 3.1232 - val_accuracy: 0.3194
Epoch 3/500
56/56 [==============================] - 7s 127ms/step - loss: 2.5872 - accuracy: 0.5089 - val_loss: 3.0383 - val_accuracy: 0.4162
Epoch 4/500
56/56 [==============================] - 7s 120ms/step - loss: 2.0587 - accuracy: 0.5870 - val_loss: 2.9381 - val_accuracy: 0.4104
Epoch 5/500
56/56 [==============================] - 7s 125ms/step - loss: 1.7335 - accuracy: 0.6152 - val_loss: 3.0389 - val_accuracy: 0.4133
Epoch 6/500
56/56 [==============================] - 7s 123ms/step - loss: 1.5809 - accuracy: 0.6470 - val_loss: 3.1035 - val_accuracy: 0.4133
Epoch 7/500
56/56 [==============================] - 7s 118ms/step - loss: 1.4126 - accuracy: 0.6669 - val_loss: 3.1645 - val_accuracy: 0.4191
Epoch 8/5

In [18]:
my_model = load_model('bestmodel.hdf5')

In [19]:
score = my_model.evaluate(xtest, ytest)
print('Test accuracy:', score[1])

22/22 [==============================] - 1s 39ms/step - loss: 2.9381 - accuracy: 0.4104
Test accuracy: 0.41040462255477905
